# Agentic AI: From Chatbots to Agents
**Presented by:** Sharad Rajore | **Organization:** Zensar Technologies

---

### 🎯 Learning Objectives
1. **Witness the Limitations:** See where a raw LLM fails (Math & Real-time data).
2. **Build Tools:** Create "Hands" for the AI using Python functions.
3. **The ReAct Loop:** Observe the *Thought -> Action -> Observation* cycle in real-time.

### 🛠️ Prerequisites
- Python 3.10+
- LangChain installed (`pip install langchain langchain-openai`)
- OpenAI API Key (or similar)

In [11]:
# 1. Setup Environment
import os
import getpass

# Enter your API Key securely
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")

## Part 1: The "Brain in a Jar" (LLM Limitations)
First, let's look at a standard LLM. It is powerful but isolated. It cannot access the outside world and often hallucinates on complex math.

In [7]:
from langchain_openai import ChatOpenAI

# Initialize the "Brain" (LLM)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# TEST 1: The Math Trap
# LLMs predict the next word, they don't calculate. 
# For large numbers, they often guess.
query = "What is 49,823 multiplied by 19,283?"

response = llm.invoke(query)
print(f"User: {query}")
print(f"LLM Answer: {response.content}")

# Reality Check: 49823 * 19283 = 960,736,909
# Check if the LLM got it right (It often fails or is slightly off without a tool).

User: What is 49,823 multiplied by 19,283?
LLM Answer: The product of 49,823 and 19,283 is 960,418,309.


## Part 2: Giving the Brain "Hands" (Defining Tools)
To fix this, we don't train the model more. We give it a **Tool**. 
In LangChain, a tool is just a Python function with a `@tool` decorator.

In [8]:
from langchain_core.tools import tool

# 1. Define the Calculator Tool
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two integers and returns the result."""
    return a * b

# 2. Define a "Mock" Search Tool (Simulating Real-time data)
@tool
def get_stock_price(ticker: str) -> str:
    """Returns the current stock price for a given ticker symbol. 
    Use 'ZENSAR' for Zensar Technologies, 'GOOGL' for Google."""
    # In a real agent, this would call Yahoo Finance API
    ticker_upper = ticker.upper().strip()
    if "ZENSAR" in ticker_upper:
        return "750.00 INR"
    elif "GOOGL" in ticker_upper or "GOOGLE" in ticker_upper:
        return "175.00 USD"
    else:
        return f"Unknown Ticker: {ticker}"

# List of tools available to our Agent
tools = [multiply, get_stock_price]

print("Tools Created: Calculator (Hands) & Stock Search (Eyes)")

Tools Created: Calculator (Hands) & Stock Search (Eyes)


## Part 3: The Agent (ReAct Loop)
Now we bind the tools to the LLM. The LLM will now act as a router:
1. **Thought:** Do I know the answer? No.
2. **Action:** Pick the right tool.
3. **Observation:** Get the output.
4. **Final Answer:** Summarize for the user.

In [9]:
from langchain.agents import create_agent

# Create a ReAct agent using the modern LangChain API
# debug=True enables verbose output showing the Thought -> Action -> Observation loop
agent_executor = create_agent(llm, tools, debug=True)

print("Agent created successfully with ReAct capabilities!")

Agent created successfully with ReAct capabilities!


## Part 4: Agent in Action
Let's ask the exact same math question, plus a question about Zensar's stock.

In [10]:
print("--- SCENARIO 1: MATH ---")
result1 = agent_executor.invoke({"messages": [("user", "What is 49,823 multiplied by 19,283?")]})
print(f"\nFinal Answer: {result1['messages'][-1].content}")

print("\n\n--- SCENARIO 2: REAL-TIME DATA ---")
result2 = agent_executor.invoke({"messages": [("user", "What is the current stock price of Zensar?")]})
print(f"\nFinal Answer: {result2['messages'][-1].content}")

--- SCENARIO 1: MATH ---
[values] {'messages': [HumanMessage(content='What is 49,823 multiplied by 19,283?', additional_kwargs={}, response_metadata={}, id='1158c90b-4df7-434e-bfde-7f5ba5acdda8')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 109, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_689bad8e9a', 'id': 'chatcmpl-CgVndrnAZ8wkQPcCKh9UqMoibvuKs', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--4a3eddb0-9395-4115-b377-a23f645632f8-0', tool_calls=[{'name': 'multiply', 'args': {'a': 49823, 'b': 19283}, 'id': 'call_UGk914VcsyCX6DPvXEaQuNyt', 'type': 'tool_

### 🧠 What just happened?
1. **Math:** Instead of guessing, you should see the agent say `Invoking: multiply`. It used the calculator.
2. **Stock:** Instead of saying "I don't know", it said `Invoking: get_stock_price`. 

**This is the shift from Generative AI (guessing) to Agentic AI (executing).**